In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/train.csv
/kaggle/input/dataset/test.csv
/kaggle/input/dataset/train/234.jpg
/kaggle/input/dataset/train/287.jpg
/kaggle/input/dataset/train/87.jpg
/kaggle/input/dataset/train/404.jpg
/kaggle/input/dataset/train/167.jpg
/kaggle/input/dataset/train/257.jpg
/kaggle/input/dataset/train/163.jpg
/kaggle/input/dataset/train/148.jpg
/kaggle/input/dataset/train/251.jpg
/kaggle/input/dataset/train/184.jpg
/kaggle/input/dataset/train/99.jpg
/kaggle/input/dataset/train/433.jpg
/kaggle/input/dataset/train/90.jpg
/kaggle/input/dataset/train/475.jpg
/kaggle/input/dataset/train/76.jpg
/kaggle/input/dataset/train/468.jpg
/kaggle/input/dataset/train/73.jpg
/kaggle/input/dataset/train/501.jpg
/kaggle/input/dataset/train/379.jpg
/kaggle/input/dataset/train/310.jpg
/kaggle/input/dataset/train/212.jpg
/kaggle/input/dataset/train/418.jpg
/kaggle/input/dataset/train/106.jpg
/kaggle/input/dataset/train/428.jpg
/kaggle/input/dataset/train/42.jpg
/kaggle/input/dataset/train/111.jpg
/kaggle

In [6]:

path= '/kaggle/input/dataset/'
k= '/kaggle/input/dataset/train.csv'
p ='/kaggle/input/dataset/test.csv'

In [7]:
train=pd.read_csv(k)
test=pd.read_csv(p)

In [8]:
train.head(5),test.head(5)

(     Image         target
 0   96.jpg       manipuri
 1  163.jpg  bharatanatyam
 2  450.jpg         odissi
 3  219.jpg      kathakali
 4  455.jpg         odissi,
      Image
 0  508.jpg
 1  246.jpg
 2  473.jpg
 3  485.jpg
 4  128.jpg)

In [9]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob

In [15]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
path ='/kaggle/input/dataset/train/'
tfms = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False,xtra_tfms=[cutout()])
data = (ImageList.from_csv(path, csv_name = '../train.csv')
        .split_by_rand_pct()              
        .label_from_df()            
        .add_test_folder(test_folder = '../test')              
        .transform(tfms, size=400)
        .databunch(num_workers=0,bs=8))

FileNotFoundError: [Errno 2] File /kaggle/input/DataSet/Train Images/../train.csv does not exist: '/kaggle/input/DataSet/Train Images/../train.csv'

In [14]:
## to see the images in train with there labels
data.show_batch(rows=3, figsize=(8,10))

AssertionError: Torch not compiled with CUDA enabled

In [ ]:

## print the target classes
print(data.classes)

In [ ]:
import torchvision.models as models
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()

In [ ]:
import torchvision.models as models
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

In [ ]:
learn = cnn_learner(data, models.resnext50_32x4d, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:

from fastai import *
from fastai.vision import *
from sklearn.model_selection import StratifiedKFold
from pathlib import Path
import shutil
from sklearn.metrics import f1_score, confusion_matrix

np.random.seed(1786)
ROOT = "/tmp/data0125004"

In [ ]:

def read_data(root):
    train_df = pd.read_csv(k)
    test_df = pd.read_csv(p)
    return train_df, test_df

In [ ]:
class SaveBestModel(Recorder):
    def __init__(self, learn,name='best_model'):
        super().__init__(learn)
        self.name = name
        self.best_loss = None
        self.best_acc = None
        self.save_method = self.save_when_acc
        
    def save_when_acc(self, metrics):        
        loss, acc = metrics[0], metrics[1]
        if (self.best_acc is None) or (acc > self.best_acc) or (loss < self.best_loss):
            self.best_acc = acc
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Save the best acc {:.5f}".format(self.best_acc))
        elif acc == self.best_acc and  loss < self.best_loss:
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Acc is eq,Save the lower loss {:.5f}".format(self.best_loss))
            
    def on_epoch_end(self,last_metrics=MetricsList,**kwargs:Any):
        self.save_method(last_metrics)

In [ ]:
#Path(ROOT).mkdir(exist_ok=True, parents=True)
src1 = "/kaggle/input/dataset/"
shutil.copytree(src1, ROOT)


if __name__=="__main__":
    train_df, test_df = read_data(ROOT)
    print(train_df.shape, test_df.shape)
    
    cvlist = list(StratifiedKFold(16, random_state=12345786).split(train_df, train_df.target))
    
    tfms1 = get_transforms(max_zoom=1.5)
    test_preds_all = []
    val_preds_all = []
    for i in range(1):
        print("Starting fold {}".format(i))
        tr_idx, val_idx = cvlist[i]
        print(tr_idx.shape, val_idx.shape)
        src = (ImageList.from_df(train_df, path=ROOT, folder="train").split_by_idxs(tr_idx, val_idx)
                                                            .label_from_df())
        data = ImageDataBunch.create_from_ll(src, ds_tfms=tfms1, size=224, bs=32, resize_method=3).normalize(imagenet_stats)
        data.add_test(ImageList.from_df(test_df, path=ROOT, folder="test"))
        learn = cnn_learner(data, models.densenet161, metrics=accuracy, ps=0.5)
        # learn.model[0].load_state_dict(torch.load("../input/save-body-weights-marvel/bestmodel_body.path"))
        cb = SaveBestModel(learn, name="bestmodel_{}".format(i))
        # learn.fit(1, callbacks=cb)
        learn.fit_one_cycle(4)
        learn.unfreeze()
        learn.fit_one_cycle(10, max_lr=1e-4, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=5e-5, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=1e-5, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=5e-6, callbacks=[cb])
        learn.fit_one_cycle(1, max_lr=1e-6, callbacks=[cb])
        learn.load("bestmodel_{}".format(i))
        val_preds, y = learn.TTA(ds_type=DatasetType.Valid)
        val_preds = np.exp(val_preds.numpy())
        print("F1 score for this fold ",f1_score(y.numpy(), np.argmax(val_preds,axis=1), average='weighted'))
        test_preds = np.exp(learn.TTA(ds_type=DatasetType.Test)[0].numpy())
        test_preds_all.append(test_preds)
        val_preds_all.append(val_preds)
        fname = "bestmodel_{}.pth".format(i)
        src = str(Path(ROOT) / "models" / fname)
        shutil.copy(src, fname)
    test_preds_all = np.mean(test_preds_all, axis=0)
    val_preds_all = np.concatenate(val_preds_all, axis=0)

    np.save("test_preds.npy", test_preds_all)
    np.save("val_preds.npy", val_preds_all)

In [ ]:
sub = test_df[["Image"]]
sub["target"] = np.argmax(test_preds, axis=1)
sub.to_csv("dense00.csv", index=False)